Description

# Configuration

In [29]:
import sys
sys.path.append('/lhome/ific/c/ccortesp/Analysis')

from libs import crudo
from libs import fit_functions as ff
from libs import plotting_tools as pt

from invisible_cities.reco.corrections import read_maps                   # type: ignore
from invisible_cities.reco.corrections import apply_all_correction        # type: ignore
from invisible_cities.types.symbols    import NormStrategy                # type: ignore
import glob
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
import matplotlib.pyplot as plt                                             # type: ignore  
import numpy as np                                                          # type: ignore
import os
import pandas as pd                                                         # type: ignore
from PIL import Image                         # type: ignore

# Styling Plot
pt.ccortesp_plot_style()
PRELIM_LOGO = np.asarray(Image.open('/lhome/ific/c/ccortesp/Analysis/images/next_logo_preliminary.png'))

%matplotlib inline
%load_ext autoreload
%autoreload 2


# ----- Extra Imports ----- #
# import logging
# import warnings
# from scipy.optimize import curve_fit
# from scipy.special import erf
# from scipy.stats import linregress
# from invisible_cities.io.dst_io               import load_dst, load_dsts
# from krcal.NB_utils.fit_energy_functions      import fit_energy
# from krcal.NB_utils.plt_energy_functions      import plot_fit_energy, print_fit_energy
# from invisible_cities.reco import corrections
from invisible_cities.reco.corrections        import read_maps

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Juanda Analysis

In [48]:
# ----- Directory Paths ----- #
BASE_PATH = '/lustre/ific.uv.es/prj/gl/neutrinos/users/villamil/kr_next100/HE'
NEXT_DATA = '/lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/Sophronia/Th_runs/15589/ldc1'
SOPH_PATH = 'run_15589_0171_ldc1_trg2.v2.3.1.20250708.HEDesman.sophronia.h5'

TEMP_DIR = '/lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/Sophronia/Low_background'
TEMP_PATH = 'run_15737_ldc1_trg2_sophronia.h5'

CUMI_PATH = '/lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/Th_analysis/h5'

# ----- Keys ----- #
KDST_KEY = 'FinalFeatures'      # Event-level
DST_KEY  = 'DST/Events'
RECO_KEY = 'RECO/Events'
EVT_KEY  = 'Events'

# ----- Runs ----- #
# RUN_IDS = [15589, 15590, 15591, 15593, 15594, 15596]
RUN_IDS = [15589]

In [49]:
# test_df = pd.read_hdf(os.path.join(NEXT_DATA, SOPH_PATH), key=RECO_KEY)
# test_df = pd.read_hdf(os.path.join(TEMP_DIR, TEMP_PATH), key=RECO_KEY)
test_df = pd.read_hdf(os.path.join(CUMI_PATH, 'processed_run_15589_ldc1_n5.h5'), key='Events')

In [50]:
test_df

,event,npeak,time,X_bary,Y_bary,Z_bary,E_final,Z_min,Z_max,R_max
0,29,39,1.751990e+09,113.750115,-387.675056,722.255106,310907.128699,652.442036,835.121603,487.977096
1,120,32,1.751990e+09,264.285986,-140.796638,572.666062,365589.157864,510.001729,651.456801,442.724204
2,309,25,1.751990e+09,-300.654627,160.416797,775.848900,503802.965524,601.435041,1012.417084,488.068255
3,358,27,1.751990e+09,-285.574259,-247.434254,178.414031,82777.347934,169.475125,192.710863,462.412582
4,393,27,1.751990e+09,55.884022,126.730073,718.332024,436580.570082,618.187063,807.841773,460.597646
5,400,27,1.751990e+09,-121.245637,-59.790596,1066.539947,201716.656165,1042.557036,1100.854576,239.562114
6,442,32,1.751990e+09,140.153177,42.734445,791.015772,68365.078801,776.658956,831.505470,205.623044
7,477,17,1.751990e+09,83.307411,-298.929018,411.818994,350277.268692,331.814000,503.986952,470.463252
8,582,20,1.751990e+09,158.078150,281.422535,883.255980,393753.855925,681.199178,964.437264,494.479703
9,687,21,1.751990e+09,211.630468,-219.259019,708.279305,354210.147939,643.954656,774.799746,392.766357


In [6]:
# Load kDSTs
all_dfs = []

for run_id in RUN_IDS:

    kdst_input_glob = f"{BASE_PATH}/{run_id}/sophronia/final_dst/*final_features_with_TE_v3.h5"
    # kdst_input_glob = f"{BASE_PATH}/{run_id}/sophronia/clean_df/*clean.h5"
    kdst_files = sorted(glob.glob(kdst_input_glob))
    if not kdst_files:
        raise FileNotFoundError(f"No kDST files found at: {kdst_input_glob}")
    
    # print(f"Found {len(kdst_files)} files for Run {run_id}: {kdst_files}")
    all_dfs.extend(pd.read_hdf(path, key=KDST_KEY) for path in kdst_files)

full_fil_df = pd.concat(all_dfs, ignore_index=True)


# print(f"Loaded {full_fil_df['event'].nunique()} unique events from kDSTs.")
# full_fil_df = full_fil_df[full_fil_df.E_final > 0]
# full_fil_df = full_fil_df[full_fil_df.DT > 0]
# full_fil_df = full_fil_df[in_range(full_fil_df.Rmax, 0, 450)]
# full_fil_df = full_fil_df[
#     in_range(full_fil_df.Zmin, 20, 1184.185) & 
#     in_range(full_fil_df.Zmax, 20, 1184.185)
# ]

# print(f"Loaded {full_fil_df['event'].nunique()} unique events from kDSTs.")

In [7]:
full_fil_df

,event,time,S1w,S1h,S1e,S1t,S2w,S2h,S2e,S2q,...,Y_bary,Z_bary,Xmax_final,Ymax_final,Zmax_final,Xmin_final,Ymin_final,Zmin_final,Rmax_final,Zdiff_final
0,29,1.751990e+09,650.0,70.675652,391.903259,604150.0,510.125,4855.565430,284933.000000,118341.734375,...,-394.267280,831.755059,227.675,-247.675,965.458500,10.975,-479.925,754.268250,336.420587,211.190250
1,120,1.751990e+09,675.0,56.735172,335.559998,790500.0,325.700,5438.912598,347422.250000,104431.320312,...,-140.575041,663.460993,381.175,13.675,753.129250,-19.125,-263.725,589.597375,381.420223,163.531875
2,309,1.751990e+09,400.0,94.684158,552.621704,696125.0,566.700,5074.413086,452668.531250,156294.125000,...,161.034777,890.956548,366.625,353.275,1170.424375,-451.525,14.175,695.300625,509.133692,475.123750
3,358,1.751990e+09,275.0,10.520331,60.535091,1199650.0,110.100,6464.333008,77638.617188,23629.761719,...,-247.208627,207.092947,-220.275,-185.975,222.787125,-359.225,-324.925,195.925000,288.284194,26.862125
4,393,1.751990e+09,750.0,78.846909,466.369110,615800.0,468.450,5077.400879,419810.875000,134321.562500,...,118.527810,835.326334,443.375,338.225,933.921125,-158.075,-185.975,714.667125,557.653603,219.254000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346348,3846990,1.752077e+09,700.0,80.077065,443.319672,525475.0,317.525,5292.683105,366544.718750,100983.171875,...,111.572019,893.852252,304.425,199.275,976.073000,-3.575,-0.875,712.886000,363.847642,263.187000
346349,3847018,1.752077e+09,300.0,13.777174,72.906906,1054150.0,131.025,6003.708496,83268.867188,28278.976562,...,126.699093,356.328643,243.225,183.725,370.293500,119.825,60.325,343.551000,304.816791,26.742500
346350,3847116,1.752077e+09,375.0,9.797277,63.984344,944025.0,131.000,4242.674316,70154.390625,26688.017578,...,88.792940,466.017242,26.525,137.075,479.248125,-112.425,29.225,452.529875,139.617804,26.718250
346351,3847130,1.752077e+09,300.0,18.348984,95.319832,701125.0,222.225,5103.366699,84164.617188,43307.468750,...,18.691254,708.910253,304.425,75.875,726.479000,119.825,-47.525,695.360625,313.738101,31.118375


In [34]:
tag_hits_df = test_df.copy()

In [ ]:
# STEP B: Redistribute energy from isolated hits to the non-isolated hits
isolated_hits_df     = tag_hits_df[tag_hits_df['is_isolated']].copy()
non_isolated_hits_df = tag_hits_df[~tag_hits_df['is_isolated']].copy()
non_isolated_hits_df['E_final'] = non_isolated_hits_df['E_corr']        # Final energy column

if not isolated_hits_df.empty and not non_isolated_hits_df.empty:

    # Z-range of the track for each event
    z_ranges = non_isolated_hits_df.groupby('event')['Z'].agg(['min', 'max']).rename(columns={'min': 'Z_min', 'max': 'Z_max'})
    # Correlate isolated hits with their event's Z-range
    isolated_in_z_range = isolated_hits_df.merge(z_ranges, on='event', how='left')
    isolated_in_z_range.dropna(subset=['Z_min', 'Z_max'], inplace=True)     # Discard hits from events without non-isolated hits

    print(isolated_in_z_range)

    # Geometric discrimnation: Z-cut
    z_cut_mask = isolated_in_z_range['Z'].between(isolated_in_z_range['Z_min'], isolated_in_z_range['Z_max'])
    kept_iso_hits = isolated_in_z_range[z_cut_mask].copy()

    print(kept_iso_hits)

    # # Energy discrimnation: from the signal/background study ---> Martin?
    # # Apply min and max values for energy of the isolated hits to be kept
    # energy_min_signal = 50.0   # Lower bound of the positive peak
    # energy_max_signal = 1000.0
    # energy_cut_mask = kept_iso_hits['E'].between(energy_min_signal, energy_max_signal)
    # kept_iso_hits = kept_iso_hits[energy_cut_mask]

    # print(kept_iso_hits)

    if not kept_iso_hits.empty:
        # Perform redistribution: QUE SEA UNA FUNCIÓN!
        # Sum the energy of kept isolated hits for each event
        energy_to_add = kept_iso_hits.groupby('event')['E_corr'].sum().rename('E_iso_to_add')

        # Now let's work on non_isolated hits
        non_isolated_hits_df = non_isolated_hits_df.merge(energy_to_add, on='event', how='left')
        non_isolated_hits_df['E_iso_to_add'].fillna(0, inplace=True)        # Events without useful isolated hits get 0 to add

        # print(merged_df)

        # Calculate the sum of non-iso energy per event to find proportions
        total_non_iso_energy = non_isolated_hits_df.groupby('event')['E_corr'].transform('sum').replace(0, 1)

        print(total_non_iso_energy)

        # Calculate proportional redistribution
        non_isolated_hits_df['E_final'] = non_isolated_hits_df['E_corr'] + \
                                          (non_isolated_hits_df['E_corr'] / total_non_iso_energy) * non_isolated_hits_df['E_iso_to_add']

      event          time  npeak        X        Y          DT          Q  \
0        29  1.751990e+09     39 -189.175 -155.375  701.405875   7.135255   
1        29  1.751990e+09     39  273.325  198.775  701.405875   7.708335   
2        29  1.751990e+09     39  381.175  137.575  705.116375   8.119436   
3        29  1.751990e+09     39 -173.625 -263.225  736.792375   7.477922   
4        29  1.751990e+09     39   -3.575 -325.425  754.268250   7.946717   
...     ...           ...    ...      ...      ...         ...        ...   
2888   3354  1.751990e+09     29  304.425 -371.575  277.332750   7.445537   
2889   3354  1.751990e+09     29 -173.625  414.975  285.775250   7.503867   
2890   3354  1.751990e+09     29  -81.325  137.075  289.418125   7.494672   
2891   3354  1.751990e+09     29  134.375  245.925  313.434000  10.616943   
2892   3354  1.751990e+09     29  196.575 -294.325  317.298750   9.469697   

               E      E_corr           Z  is_isolated       Z_min       Z_m

In [28]:
kept_iso_hits

,event,time,npeak,X,Y,DT,Q,E,E_corr,Z,is_isolated,Z_min,Z_max
4,29,1.751990e+09,39,-3.575,-325.425,754.268250,7.946717,450.705870,488.544161,652.442036,True,652.442036,835.121603
5,29,1.751990e+09,39,119.825,-386.625,754.268250,7.669684,434.993680,494.628435,652.442036,True,652.442036,835.121603
6,29,1.751990e+09,39,-81.325,13.675,758.117625,9.360104,179.473454,184.496284,655.771746,True,652.442036,835.121603
7,29,1.751990e+09,39,-19.125,-417.725,761.923125,11.270300,126.528589,148.703683,659.063503,True,652.442036,835.121603
8,29,1.751990e+09,39,288.875,-185.975,761.923125,7.062734,79.291388,84.347769,659.063503,True,652.442036,835.121603
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861,3354,1.751990e+09,29,335.525,276.525,77.530125,7.998031,67.554320,79.519962,67.063558,True,61.054187,83.894944
2862,3354,1.751990e+09,29,288.875,337.725,77.530125,9.044462,76.392862,93.560070,67.063558,True,61.054187,83.894944
2863,3354,1.751990e+09,29,458.925,-1.375,77.530125,8.556113,72.268087,89.968545,67.063558,True,61.054187,83.894944
2864,3354,1.751990e+09,29,257.775,307.625,85.246375,7.467422,61.743574,70.779799,73.738114,True,61.054187,83.894944


In [40]:
merged_df

,event,time,npeak,X,Y,DT,Q,E,E_corr,Z,is_isolated,E_final,E_iso_to_add
0,29,1.751990e+09,39,57.625,-433.275,754.268250,12.095537,686.010265,844.721284,652.442036,False,844.721284,7611.302968
1,29,1.751990e+09,39,57.625,-417.725,754.268250,11.118161,630.577427,733.293862,652.442036,False,733.293862,7611.302968
2,29,1.751990e+09,39,104.275,-402.175,754.268250,7.684647,435.842276,506.535269,652.442036,False,506.535269,7611.302968
3,29,1.751990e+09,39,26.525,-433.275,758.117625,8.115583,155.610648,191.692142,655.771746,False,191.692142,7611.302968
4,29,1.751990e+09,39,26.525,-417.725,758.117625,23.911102,458.478697,534.951479,655.771746,False,534.951479,7611.302968
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33288,3354,1.751990e+09,29,227.675,230.375,93.316750,17.072540,281.077304,300.175915,80.718989,False,300.175915,1896.137956
33289,3354,1.751990e+09,29,227.675,245.925,93.316750,36.787003,605.650439,657.247392,80.718989,False,657.247392,1896.137956
33290,3354,1.751990e+09,29,243.225,230.375,93.316750,12.343800,203.224702,222.158228,80.718989,False,222.158228,1896.137956
33291,3354,1.751990e+09,29,243.225,245.925,93.316750,14.567748,239.839139,265.657581,80.718989,False,265.657581,1896.137956


In [20]:
non_isolated_hits_df

,event,time,npeak,X,Y,DT,Q,E,E_corr,Z,is_isolated,E_final
5,29,1.751990e+09,39,57.625,-433.275,754.268250,12.095537,686.010265,844.721284,652.442036,False,844.721284
6,29,1.751990e+09,39,57.625,-417.725,754.268250,11.118161,630.577427,733.293862,652.442036,False,733.293862
7,29,1.751990e+09,39,104.275,-402.175,754.268250,7.684647,435.842276,506.535269,652.442036,False,506.535269
10,29,1.751990e+09,39,26.525,-433.275,758.117625,8.115583,155.610648,191.692142,655.771746,False,191.692142
11,29,1.751990e+09,39,26.525,-417.725,758.117625,23.911102,458.478697,534.951479,655.771746,False,534.951479
...,...,...,...,...,...,...,...,...,...,...,...,...
36154,3354,1.751990e+09,29,227.675,230.375,93.316750,17.072540,281.077304,300.175915,80.718989,False,300.175915
36155,3354,1.751990e+09,29,227.675,245.925,93.316750,36.787003,605.650439,657.247392,80.718989,False,657.247392
36156,3354,1.751990e+09,29,243.225,230.375,93.316750,12.343800,203.224702,222.158228,80.718989,False,222.158228
36157,3354,1.751990e+09,29,243.225,245.925,93.316750,14.567748,239.839139,265.657581,80.718989,False,265.657581


In [43]:
non_isolated_hits_df

,event,time,npeak,X,Y,DT,Q,E,E_corr,Z,is_isolated,E_final
5,29,1.751990e+09,39,57.625,-433.275,754.268250,12.095537,686.010265,844.721284,652.442036,False,186.434438
6,29,1.751990e+09,39,57.625,-417.725,754.268250,11.118161,630.577427,733.293862,652.442036,False,619.535372
7,29,1.751990e+09,39,104.275,-402.175,754.268250,7.684647,435.842276,506.535269,652.442036,False,598.177642
10,29,1.751990e+09,39,26.525,-433.275,758.117625,8.115583,155.610648,191.692142,655.771746,False,330.384226
11,29,1.751990e+09,39,26.525,-417.725,758.117625,23.911102,458.478697,534.951479,655.771746,False,743.772340
...,...,...,...,...,...,...,...,...,...,...,...,...
36154,3354,1.751990e+09,29,227.675,230.375,93.316750,17.072540,281.077304,300.175915,80.718989,False,NaN
36155,3354,1.751990e+09,29,227.675,245.925,93.316750,36.787003,605.650439,657.247392,80.718989,False,NaN
36156,3354,1.751990e+09,29,243.225,230.375,93.316750,12.343800,203.224702,222.158228,80.718989,False,NaN
36157,3354,1.751990e+09,29,243.225,245.925,93.316750,14.567748,239.839139,265.657581,80.718989,False,NaN


In [44]:
E309_df = non_isolated_hits_df[non_isolated_hits_df['event'] == 309]

In [47]:
E309_df['E_final'].sum()

532905.6499537922